In [2]:
import sys

sys.path.append('..')

In [1]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt

# no line wrap
pd.set_option('display.expand_frame_repr', False)

data = 'human'
channels = '*'
limit = 10000
batch_size = 64
lrs = [1e-3, 5e-4, 1e-4]
weight_decays = [0, 1e-4]
epochs = 20

# Define the directory and file pattern
# name = f'{model_name}_{args.data}_{args.channels}_{args.limit}_{args.num_moves}_{args.engine_prob}_{args.batch_size}_{args.lr}_{args.weight_decay}_{args.epochs}.csv'
# example = ConvLSTM_synthetic_12_10000_60_0.5_64_0.001_0.0001_20.csv
# pattern = os.path.join('../results', '*_processed_6_10000_40_*_64_*_*.csv')
pattern = os.path.join('../results', f'*_{data}_{channels}*.csv')
files = glob.glob(pattern)

results = {}


for file in files:
  data = file.split('/')[-1].split('_')
  model = data[0]
  lr = float(data[-3])
  weight_decay = float(data[-2])
  channels = int(data[2])
  df = pd.read_csv(file)
  df = df[df['type'] == 'test']
  row = df[df['accuracy'] == df['accuracy'].max()].iloc[0].to_dict()
  del row['epoch']
  del row['loss']
  del row['type']
  row['lr'] = lr
  row['wd'] = weight_decay
  row['channels'] = int(channels)
  if model not in results:
    results[model] = row
  else:
    if row['accuracy'] > results[model]['accuracy']:
      results[model] = row

df = pd.DataFrame(results).T
df = df.sort_values('accuracy', ascending=False)
print(df)

                           accuracy precision    recall        f1       auc    dataset       lr      wd channels
ConvLSTM(times)                 0.6  0.585518       0.6  0.466854  0.496375  processed    0.001  0.0001       12
Transformer(times)         0.595455  0.358904  0.595455  0.447863  0.503795  processed  0.00005  0.0001       12
ConvLSTM(evals, times)     0.577273  0.356075  0.577273  0.440462  0.417678  processed   0.0005  0.0001       12
Transformer(evals, times)  0.577273   0.50544  0.577273   0.47385  0.484971  processed    0.001     0.0       12
Transformer(evals)         0.572727  0.496899  0.572727  0.486242  0.506774  processed    0.001  0.0001       12
ConvLSTM(evals)            0.572727  0.394844  0.572727  0.445468  0.430138  processed    0.001     0.0        6
ConvLSTM                   0.563636  0.474845  0.563636  0.487656  0.536457  processed   0.0005  0.0001       12
Transformer                0.540909  0.444381  0.540909  0.473507   0.49621  processed   0.0001 